In [ ]:
!pip install gradio icecream langchain openai sentence-transformers tiktoken langchain-mongodb langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.5 MB/

In [ ]:
!python -m pip install "pymongo[srv]"

In [ ]:
# Import libraries.
import getpass
import gradio as gr
import os
import pprint
import requests
import sys

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from gradio.themes.base import Base
from google.colab import drive
from icecream import ic
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import MongoDBAtlasVectorSearch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["MONGO_URI"] = getpass.getpass();

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MONGO_URI = os.getenv('MONGO_URI')

··········
··········


In [ ]:
# mongodb client init
mclient = MongoClient(MONGO_URI, server_api=ServerApi('1'))

In [ ]:
# Connect to MongoDB Atlas cluster using the connection string.
cluster = MongoClient(MONGO_URI)

# Define the MongoDB database and collection name.
DB_NAME = "blackrag"
COLLECTION_NAME = "blackjack"

# Connect to the specific collection in the database.
MONGODB_COLLECTION = cluster[DB_NAME][COLLECTION_NAME]

vector_search_index = "vector_index"

In [ ]:
MONGODB_COLLECTION.delete_many({})

DeleteResult({'n': 875, 'electionId': ObjectId('7fffffff00000000000001f5'), 'opTime': {'ts': Timestamp(1715111576, 909), 't': 501}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1715111576, 914), 'signature': {'hash': b'va\xc0\x94\xc0\xe2M\xfd\xb0i\xc1\xbaX\xfa)\\B\x19\x1el', 'keyId': 7327356035386048545}}, 'operationTime': Timestamp(1715111576, 909)}, acknowledged=True)

In [ ]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
directory_path = "/content/drive/MyDrive/blackjack_rag"

sys.path.append(directory_path)

In [ ]:
my_txts = os.listdir(directory_path)
my_txts

['Ultimate BlackJack.txt',
 "Don Schlesinger - Blackjack Attack_ Playing the Pros' Way-RGE Publications (1997).txt"]

In [ ]:
loaders = []
for my_txt in my_txts:
  my_txt_path = os.path.join(directory_path, my_txt)
  loaders.append(TextLoader(my_txt_path))

loaders

In [ ]:
data = []
for loader in loaders:
  data.append(loader.load())

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150)

In [ ]:
docs = []
for doc in data:
  chunk = text_splitter.split_documents(doc)
  docs.append(chunk)

In [ ]:
# Debugging purposes.
# Print the number of total documents to be stored in the vector database.
total = 0
for i in range(len(docs)):
  if i == len(docs) - 1:
    print(len(docs[i]), end="")
  else:
    print(len(docs[i]), "+ " ,end="")
  total += len(docs[i])
print(" =", total, " total documents\n")

# Print the first document.
print(docs[0], "\n\n\n")

# Print the total number of PDF files.
# docs is a list of lists where each list stores all the documents for one PDF file.
print(len(docs))

51 + 824 = 875  total documents

[Document(page_content='Rules \nThe game itself is simple. You, the player, attempt to accumu\x02late cards with a numerical total closer to (but not more than) \ntwenty-one than those accumulated by the dealer. If you do so, \nyou win. If the dealers\' total is closer to twenty-one than \nyours, you lose. Winning hands are paid off at even odds. If you \nand the dealer both arrive at the same total, the hand is a \n"push," and nobody wins. All bets must be made before any cards \nare dealt, and no bet may be changed once the first card has \nbeen dealt. \nEach player is initially dealt two cards; they may be face -down \nor face-up, depending on the rules of the casino. The dealer \ngets two cards, one face-up and one face-down. The value of the', metadata={'source': '/content/drive/MyDrive/blackjack_rag/Ultimate BlackJack.txt'}), Document(page_content="or face-up, depending on the rules of the casino. The dealer \ngets two cards, one face-up and one f

In [ ]:
merged_documents = []

for doc in docs:
  merged_documents.extend(doc)

# Print the merged list of all the documents.
print("len(merged_documents) =", len(merged_documents))

len(merged_documents) = 875


In [ ]:
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=merged_documents, # The sample documents to store in the vector database.
    embedding=OpenAIEmbeddings(), # OpenAI's embedding model as the model used to convert text into vector embeddings for the embedding field.
    collection=MONGODB_COLLECTION, # pdfs.pdfs_collection as the Atlas collection to store the documents.
    index_name=vector_search_index # vector_index as the index to use for querying the vector store.
)